In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [19]:
import sys
import numpy as np
import pandas as pd

sys.path.append('/lfs/1/sahaana/enrichment/ember/utils')
from preprocessing_utils import reindex_deepmatcher, MARCO_1k_BM25, merge_columns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Group records into 1k PID buckets and BM25

In [6]:
path = "/lfs/1/sahaana/enrichment/data/MSMARCO/"
merged_col = "merged_all"
separator = "[SEP]"
dataset = 'MARCO'

In [6]:
file = "collection.tsv"
collection = pd.read_csv(f"{path}/{file}",sep='\t', header=None, 
                         index_col=0, names=['PID','Passage'])
collection

/future/u/sahaana/miniconda3/envs/future9_ludwig/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Passage
PID,
0,The presence of communication amid scientific ...
1,The Manhattan Project and its atomic bomb help...
2,Essay on The Manhattan Project - The Manhattan...
3,The Manhattan Project was the name for a proje...
4,versions of each volume as well as complementa...
...,...
8841818,When metal salts emit short wavelengths of vis...
8841819,Thousands of people across the United States w...
8841820,"The recipe that creates blue, for example, inc..."


In [29]:
file = "queries.train"
queries = pd.read_csv(f"{path}/{file}.tsv",sep='\t',header=None,
                     names=['QID', 'Query'], index_col=0)
queries

,Query
QID,
121352,define extreme
634306,what does chattel mean on credit history
920825,what was the great leap forward brainly
510633,tattoo fixers how much does it cost
737889,what is decentralization process.
...,...
633855,what does canada post regulations mean
1059728,wholesale lularoe price
210839,how can i watch the day after


In [8]:
file = "top1000.train.txt"
top1000 = pd.read_csv(f"{path}/{file}",sep='\t', header=None, usecols=[0,1], names=['QID', 'PID'])
top1000

,QID,PID
0,965162,1000930
1,279558,1000930
2,279511,1000930
3,279498,1000930
4,739878,1000930
...,...,...
478002388,727506,999540
478002389,633883,999540
478002390,898757,999540
478002391,583095,999540


In [9]:
grouped_queries = top1000.groupby('QID')['PID'].apply(list)

In [17]:
grouped_queries = top1000.groupby('QID')['PID'].apply(list)
grouped_queries = pd.DataFrame(grouped_queries)

In [101]:
pd.to_pickle(grouped_queries, f"{path}/top1000.train.grouped.pkl")

In [189]:
bm25s = MARCO_1k_BM25(corpus_df = collection, 
                      query_df = queries,
                      bm25_1k = f"{path}/top1000.train.grouped.pkl",
                      grouped = True)

  0%|          | 0/501233 [00:00<?, ?it/s]

In [196]:
combined_bm25s = pd.DataFrame(np.hstack([bm25s[0], bm25s[1]]))
combined_bm25s = combined_bm25s.set_index(0)

In [197]:
pd.to_pickle(combined_bm25s, '/lfs/1/sahaana/enrichment/data/Okapi25Queries/MARCO_1k_argsort_indices.pkl')

In [4]:
pd.read_pickle('/lfs/1/sahaana/enrichment/data/Okapi25Queries/MARCO_1k_argsort_indices.pkl')

,1,2,3,4,5,6,7,8,9,10,...,992,993,994,995,996,997,998,999,1000,1001
0,,,,,,,,,,,,,,,,,,,,,
3,-1,3305827,4710431,7472388,3305829,7472384,3305830,3267804,8708593,4937072,...,6810086,3060832,6979294,3172383,5031304,6303931,1142685,6979295,1142682,3938879
4,-1,8297558,8412814,6145200,2121343,7389524,4987684,8267533,7906779,3645740,...,5528798,692880,7029522,4876159,2878039,3483260,2640288,4557675,753177,2499205
5,-1,4651118,6632988,862283,7234816,4919639,4925095,3843862,2758630,6964618,...,2162071,4956433,372423,2672924,4322870,1008848,1008857,7841693,2206805,2162073
6,-1,3625821,3132321,4003845,8758167,8678879,3188798,3188802,1168370,4190315,...,5042221,6937323,6468285,6497426,2063899,4605755,6937322,2063903,1495713,2359698
8,-1,6290676,1267580,1087028,1771174,1771171,6290677,6821664,7708477,4786613,...,975280,975281,8704236,2252437,3686190,2080059,3013032,307190,2412830,8383396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185863,-1,7485469,2465795,6639668,3971817,4337572,8701346,7676743,7931927,185044,...,629017,2600087,4530475,8654020,4628495,8654023,8654017,1817583,266301,2009361
1185864,-1,489823,2345669,7736948,5774739,4112569,3512081,4717128,4132464,1830010,...,7361462,1267422,2797946,611284,2345672,3001419,7130104,7862761,8191765,7862765
1185865,-1,6784244,7079069,7047871,7673495,6558040,8613128,7631040,7793643,2135665,...,3759152,8591552,5925773,7219780,6811511,5392448,7428387,2780686,2730860,1305705


# Construct Triplets

In [7]:
!ls "/lfs/1/sahaana/enrichment/data/MSMARCO/"


collection.tsv			queries.train.tsv
fulldocs.tsv			raw
msmarco_eval_ranking.py		tableA_processed.pkl
qidpidtriples.train.full.2.tsv	tableB_processed.pkl
qrels.dev.small.tsv		top1000.dev
qrels.dev.tsv			top1000.pidqid.pkl
qrels.train.tsv			top1000.train.grouped.pkl
queries.dev.small.tsv		top1000.train.txt
queries.dev.tsv			triples.train.full.tsv
queries.eval.small.tsv		triples.train.small.tsv
queries.eval.tsv


In [7]:
file = "qrels.dev.tsv"
dev_qrels = pd.read_csv(f"{path}/{file}",sep='\t',
                        header=None, 
                        usecols=[0,2], 
                        names=['QID', 'PID'])
dev_qrels

,QID,PID
0,1102432,2026790
1,1102431,7066866
2,1102431,7066867
3,1090282,7066900
4,39449,7066905
...,...,...
59268,150337,8009410
59269,22241,8009429
59270,129177,8009442
59271,190655,3576091


In [8]:
file = "qrels.dev.small.tsv"
dev_qrels_small = pd.read_csv(f"{path}/{file}",sep='\t',
                        header=None, 
                        usecols=[0,2], 
                        names=['QID', 'PID'])
dev_qrels_small

,QID,PID
0,300674,7067032
1,125705,7067056
2,94798,7067181
3,9083,7067274
4,174249,7067348
...,...,...
7432,147073,8008770
7433,243761,8008787
7434,162662,8008977
7435,247194,8009319


In [14]:
file = "qrels.train.tsv"
train_qrels = pd.read_csv(f"{path}/{file}",sep='\t',
                        header=None, 
                        usecols=[0,2], 
                        names=['QID', 'PID'])
train_qrels

,QID,PID
0,1185869,0
1,1185868,16
2,597651,49
3,403613,60
4,1183785,389
...,...,...
532756,19285,8841362
532757,558837,4989159
532758,559149,8841547
532759,706678,8841643


In [17]:
file = "triples.train.small.tsv"
train_triples = pd.read_csv(f"{path}/{file}",sep='\t',
                        header=None)
train_triples

,0,1,2
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."
...,...,...,...
39780806,who are the royal guards,"Talk78 Share. The Royal Guard (çå±ç¹å, Å...","A tall jar of royal jelly. Royal jelly, a secr..."
39780807,difference between chemistry and food calorie,Re: calorie vs. a food calorie. A food calorie...,"In order to answer this question, it helps to ..."
39780808,where do belgian draft horses originate,ORIGIN. The Belgian Draft Horse is descended f...,More About Biblical Name: Philip. Philip is a ...
39780809,the unit of measurement used by nutritionists ...,A unit of heat measurement used in nutrition t...,"In order to answer this question, it helps to ..."


# Process Dev Set

In [16]:
file = "queries.dev.small"
queries_dev = pd.read_csv(f"{path}/{file}.tsv",sep='\t',header=None,
                     names=['QID', 'Query'])
queries_dev

,QID,Query
0,1048585,what is paula deen's brother
1,2,Androgen receptor define
2,524332,treating tension headaches without medication
3,1048642,what is paranoid sc
4,524447,treatment of varicose veins in legs
...,...,...
6975,734979,what is coronary cta
6976,524166,transportation to us bank stadium
6977,968921,where did last names originate
6978,786375,what is preoperative clearance


In [21]:
query_path = f"{path}/dev_tableA_processed.pkl"
#corpus_path = f"{path}/tableB_processed.pkl"

query_columns = list(queries_dev.columns)
#corpus_columns = list(corpus_df.columns)

query_columns.remove('QID')
#corpus_columns.remove('PID')

query_df = merge_columns(queries_dev, query_columns, merged_col, query_path, separator)
#corpus_df = merge_columns(corpus_df, corpus_columns, merged_col, corpus_path, separator)

In [22]:
queries_dev

,QID,Query,merged_all
0,1048585,what is paula deen's brother,[SEP] Query what is paula deen's brother
1,2,Androgen receptor define,[SEP] Query Androgen receptor define
2,524332,treating tension headaches without medication,[SEP] Query treating tension headaches witho...
3,1048642,what is paranoid sc,[SEP] Query what is paranoid sc
4,524447,treatment of varicose veins in legs,[SEP] Query treatment of varicose veins in legs
...,...,...,...
6975,734979,what is coronary cta,[SEP] Query what is coronary cta
6976,524166,transportation to us bank stadium,[SEP] Query transportation to us bank stadium
6977,968921,where did last names originate,[SEP] Query where did last names originate
6978,786375,what is preoperative clearance,[SEP] Query what is preoperative clearance


In [24]:
query_path

'/lfs/1/sahaana/enrichment/data/MSMARCO//dev_tableA_processed.pkl'